In [67]:
# HElP about the coordinates can be found in https://ratschlab.slack.com/archives/DD4F40KPF/p1648498023344119 (DM LP/ AK)
# /cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5 and
# /cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle
# There, I looked for gene ENSG00000085733.16, which has ID 9803. Then I checked which edge in the segmentgraph corresponds to 70435040;70436454 , which are segments 169 (array([70435025, 70435040])) and 186 (array([70436454, 70436562])). This translates to a multi_index integer of 32296. Checking in the correct range of the hdf5 where the gene_ids_edges is 9803 for the edge_idx of 32296, gets an offset value of 175. Long story short, the edge corresponds to the count matrix entry edges[221005, :] . In this count vector only a single index has a non-zero value (index 855), which corresponds to sample TCGA-E2-A1B4-01A-11R-A12P-07.all. In this sample I could also find on the single sample level a junction count of 2 for this junctions. For all other samples (I used a bash loop), I was not able to find support for the junction.

In [76]:
# Pseudo code to get a junction expression using the graph and the count file

# 1. GET THE EDGES_INDEXES that correspond to the gene of interest VIA: 
# edges_index_gene = [i for i, j in enumerate(h5f['gene_ids_edges'][:, 0]) if j == gene_hd5_id]

# 2. GET the multi_indexes
# SLICE h5f['edge_idx'] for the EDGES_INDEXES above: 
# multi_indexes = h5f['edge_idx'][edges_index_gene] 

# 3. THE multi_indexes obained match perfectly the ones in np.where(gene.segmentgraph.seg_edges.flatten())[0]

# REMARK: HISTORICALLY I used to work more whith the offsets that the multi_indexes
# Can be seen this way
#print([ i for i, foo in enumerate(zip(np.where(gene.segmentgraph.seg_edges)[0], np.where(gene.segmentgraph.seg_edges)[1])) 
#if (foo[0] == 169) & (foo[1] == 186)]) # THIS IS THE OFFSET
# The offset is directly associated with the non-flattened structure. 

# 4. TO get an offset: 
#offset = [myid for myid, edge_idx in enumerate(h5f['edge_idx'][edges_index_gene]) if edge_idx == multi_index_to_locate][0]  
# OR 
# offset = [myid for myid, edge_idx in enumerate(multi_indexes) if edge_idx == multi_index_to_locate][0]  
# 5. We can check that h5f['edge_idx'][edges_index_gene][offset] = multi_index_to_locate

# TO GET the EDGE index of my favorite junction (absolute_pos) from the multi_index_to_locate
#absolute_pos = [myid for myid, edge_idx in enumerate(h5f['edge_idx']) if edge_idx == multi_index_to_locate][0]


# TO GET the EDGE index of my favorite junction (absolute_pos) from the offset
# edges_index_gene[offset]

# REMARK: We can check that h5f['edge_idx'][absolute_pos] = multi_index_to_locate

# TO GET the EDGE EXPRESSION of of my favorite junction: 
# h5f['edges'][absolute_pos, :] # all samples

# TO GET the EDGE EXPRESSION of all junctions for a given sample
#h5f['edges'][edges_index_gene, sample]


In [1]:
import os 
import pandas as pd
import path
import pickle 
import numpy as np
import h5py

In [2]:
sample_type = 'BRCA' # BRCA
base_path_junctions = '/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes'

In [45]:
## Spladder junction files
# OV
folder_OV_jx_unfiltered = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv'
folder_OV_jx_conf2 = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv'

# BRCA
folder_BRCA_jx_unfiltered = 'TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_unfiltered_tsv'
folder_BRCA_jx_conf2 = 'TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_conf_2_tsv'

## Spladder Graphs
# OV
count_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.rechunked.hdf5')
#count_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5')
splice_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_OV = '/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_Ov_378.tsv'

# BRCA
count_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5')
splice_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_BRCA = "/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_BRCA_1102.tsv"

In [46]:
if sample_type == 'OV':
    splice_path = splice_path_OV
    count_path = count_path_OV

    folder_jx_unfiltered = folder_OV_jx_unfiltered
    folder_jx_conf2 = folder_OV_jx_conf2

elif sample_type == 'BRCA':
    splice_path = splice_path_BRCA
    count_path = count_path_BRCA

    folder_jx_unfiltered = folder_BRCA_jx_unfiltered
    folder_jx_conf2 = folder_BRCA_jx_conf2 # TODO + GET TABLES FROM MARIIA's files

In [47]:
sample = 'sample1' # relink samples 1 to 5

In [48]:
path_jx_unfiltered = os.path.join(base_path_junctions, folder_jx_unfiltered, sample)
print(path_jx_unfiltered)
path_jx_conf2 = os.path.join(base_path_junctions, folder_jx_conf2, sample)
print(path_jx_conf2)
path_jx_unfiltered = os.path.realpath(path_jx_unfiltered)
print(path_jx_unfiltered)
path_jx_conf2 = os.path.realpath(path_jx_conf2)
print(path_jx_conf2)

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_unfiltered_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_conf_2_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_unfiltered_tsv/TCGA-C8-A12P-01A-11R-A115-07.all.junctions.tsv.gz
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_conf_2_tsv/TCGA-C8-A12P-01A-11R-A115-07.all.conf_2.filt.junctions.tsv.gz


In [50]:
sample_name = path_jx_unfiltered.split('/')[-1].replace('.junctions.tsv.gz', '')
print(sample_name)

TCGA-C8-A12P-01A-11R-A115-07.all


# "Spladder files"

In [51]:
# Load
df_jx_unfiltered = pd.read_csv(path_jx_unfiltered,
                                  sep = '\t')
df_jx_conf2 = pd.read_csv(path_jx_conf2, 
                                  sep = '\t')

In [52]:
def unique_junctions_df(df):
    display(df.head(3))
    print('shape table')
    print(df.shape)
    print('shape table excluding column count')
    print(df.drop(['count'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, chr')
    print(df.drop(['count', 'chr'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand')
    print(df.drop(['count', 'strand'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand, chr')
    print(df.drop(['count', 'strand', 'chr'], axis = 1).drop_duplicates().shape)
    

In [53]:
print(path_jx_unfiltered)
unique_junctions_df(df_jx_unfiltered)
print(path_jx_conf2)
unique_junctions_df(df_jx_conf2)

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_unfiltered_tsv/TCGA-C8-A12P-01A-11R-A115-07.all.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,47436,48113,1
1,chr10,-,48114,48424,1
2,chr10,-,48114,48614,4


shape table
(255354, 5)
shape table excluding column count
(255354, 4)
shape table excluding column count, chr
(255233, 3)
shape table excluding column count, strand
(255337, 3)
shape table excluding column count, strand, chr
(255216, 2)
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_conf_2_tsv/TCGA-C8-A12P-01A-11R-A115-07.all.conf_2.filt.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,47436,48113,1
1,chr10,-,48114,48424,1
2,chr10,-,48114,48614,4


shape table
(222813, 5)
shape table excluding column count
(222813, 4)
shape table excluding column count, chr
(222709, 3)
shape table excluding column count, strand
(222800, 3)
shape table excluding column count, strand, chr
(222696, 2)


# Graph Single Gene

In [7]:
graph, meta = pickle.load(open(splice_path, 'rb'))

In [8]:
gene = graph[9803]
gene.from_sparse()

In [9]:
gene.name

'ENSG00000085733.16'

In [23]:
gene.strand

'+'

In [10]:
gene.segmentgraph.seg_edges.shape

(190, 190)

In [11]:
gene.splicegraph.edges.shape

(205, 205)

In [12]:
#vars(gene.splicegraph)

In [13]:
gene.splicegraph.edges[0:20, 0:20]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
edges_flattened = gene.splicegraph.edges.flatten()

In [15]:
segments_flattened = gene.segmentgraph.seg_edges.flatten()

In [16]:
edges_graph_index = np.array([(a,b) for a, b in zip(np.where(gene.splicegraph.edges)[0],
                                               np.where(gene.splicegraph.edges)[1])])
print(np.array(edges_graph_index)[0:10])
print(np.array(edges_graph_index).shape)

[[ 0 20]
 [ 1  8]
 [ 1  9]
 [ 1 10]
 [ 1 12]
 [ 1 15]
 [ 1 16]
 [ 1 17]
 [ 1 18]
 [ 1 19]]
(1284, 2)


In [17]:
seg_graph_index = np.array([(a,b) for a, b in zip(np.where(gene.segmentgraph.seg_edges)[0],
                                               np.where(gene.segmentgraph.seg_edges)[1])])
print(np.array(seg_graph_index)[0:10])
print(np.array(seg_graph_index).shape)

[[ 0 21]
 [ 4  8]
 [ 4 11]
 [ 4 17]
 [ 4 18]
 [ 4 20]
 [ 4 21]
 [ 4 22]
 [ 4 23]
 [ 4 30]]
(187, 2)


In [18]:
edges_graph_index_flattened = np.where(edges_flattened)[0]
print(edges_graph_index_flattened[0:min(100, len(edges_graph_index_flattened))])

[  20  213  214  215  217  220  221  222  223  224  225  226  227  228
  229  235  252  260  262  286  287  289  418  419  420  422  425  426
  427  428  429  430  431  432  433  434  440  457  465  467  491  492
  494  623  624  625  627  630  631  632  633  634  635  636  637  638
  639  645  662  670  672  696  697  699  828  829  830  832  835  836
  837  838  839  840  841  842  843  844  850  867  875  877  901  902
  904 1045 1046 1250 1455 1456 1641 1642 1643 1644 1846 1847 1848 1849
 1860 1865]


In [19]:
segs_graph_index_flattened = np.where(segments_flattened)[0]
print(segs_graph_index_flattened[0:min(100, len(segs_graph_index_flattened))])

[   21   768   771   777   778   780   781   782   783   790   809   816
   817   844  1351  1541  2111  2677  2681  2867  2871  3441  4400  4586
  4588  4589  4590  4594  4780  5350  5924  6114  6115  6304  6699  6889
  7070  7071  7074  7079  7163  7269  8789  8790  8792  8796  8797  8979
  8980  8982  8986  8987  9360  9362  9363  9366  9367  9375  9377 10126
 10127 10506 11104 11272 11461 11462 11463 11465 11467 11469 11522 11552
 12035 12037 12038 12039 12054 12068 13194 13384 13565 13566 13570 13573
 13574 13588 13608 13756 13764 13798 13954 14144 15284 16046 16628 16629
 16631 16632 16648 16649]


In [65]:
for index, multi_index in zip(seg_graph_index, segs_graph_index_flattened):
    print(index, multi_index)

[ 0 21] 21
[4 8] 768
[ 4 11] 771
[ 4 17] 777
[ 4 18] 778
[ 4 20] 780
[ 4 21] 781
[ 4 22] 782
[ 4 23] 783
[ 4 30] 790
[ 4 49] 809
[ 4 56] 816
[ 4 57] 817
[ 4 84] 844
[ 7 21] 1351
[ 8 21] 1541
[11 21] 2111
[14 17] 2677
[14 21] 2681
[15 17] 2867
[15 21] 2871
[18 21] 3441
[23 30] 4400
[24 26] 4586
[24 28] 4588
[24 29] 4589
[24 30] 4590
[24 34] 4594
[25 30] 4780
[28 30] 5350
[31 34] 5924
[32 34] 6114
[32 35] 6115
[33 34] 6304
[35 49] 6699
[36 49] 6889
[37 40] 7070
[37 41] 7071
[37 44] 7074
[37 49] 7079
[ 37 133] 7163
[38 49] 7269
[46 49] 8789
[46 50] 8790
[46 52] 8792
[46 56] 8796
[46 57] 8797
[47 49] 8979
[47 50] 8980
[47 52] 8982
[47 56] 8986
[47 57] 8987
[49 50] 9360
[49 52] 9362
[49 53] 9363
[49 56] 9366
[49 57] 9367
[49 65] 9375
[49 67] 9377
[53 56] 10126
[53 57] 10127
[55 56] 10506
[58 84] 11104
[59 62] 11272
[60 61] 11461
[60 62] 11462
[60 63] 11463
[60 65] 11465
[60 67] 11467
[60 69] 11469
[ 60 122] 11522
[ 60 152] 11552
[63 65] 12035
[63 67] 12037
[63 68] 12038
[63 69] 12039
[63 84

In [110]:
# TEST
# ME: Yes the multi index for the segment is correct 
# [169 186] 32296

In [31]:
# CHECK ORDER
junction_gene = [(gene.splicegraph.vertices[1][i], gene.splicegraph.vertices[0][j]) 
                 for i, j in edges_graph_index]

print(junction_gene[0:10])

[(70398064, 70405264), (70398614, 70399506), (70398614, 70399506), (70398614, 70399506), (70398614, 70399597), (70398614, 70402838), (70398614, 70402838), (70398614, 70402844), (70398614, 70403210), (70398614, 70405264)]


In [32]:
segms_gene = [(gene.segmentgraph.segments[1][i], gene.segmentgraph.segments[0][j]) 
              for i, j in seg_graph_index]
print(segms_gene[0:10])

[(70398064, 70405264), (70398614, 70399506), (70398614, 70399597), (70398614, 70402838), (70398614, 70402844), (70398614, 70403210), (70398614, 70405264), (70398614, 70405265), (70398614, 70405321), (70398614, 70407297)]


In [33]:
for i, j in zip(edges_graph_index, junction_gene):
    print(i, j)

[ 0 20] (70398064, 70405264)
[1 8] (70398614, 70399506)
[1 9] (70398614, 70399506)
[ 1 10] (70398614, 70399506)
[ 1 12] (70398614, 70399597)
[ 1 15] (70398614, 70402838)
[ 1 16] (70398614, 70402838)
[ 1 17] (70398614, 70402844)
[ 1 18] (70398614, 70403210)
[ 1 19] (70398614, 70405264)
[ 1 20] (70398614, 70405264)
[ 1 21] (70398614, 70405264)
[ 1 22] (70398614, 70405265)
[ 1 23] (70398614, 70405321)
[ 1 24] (70398614, 70405321)
[ 1 30] (70398614, 70407297)
[ 1 47] (70398614, 70409830)
[ 1 55] (70398614, 70414541)
[ 1 57] (70398614, 70414545)
[ 1 81] (70398614, 70419745)
[ 1 82] (70398614, 70419745)
[ 1 84] (70398614, 70419745)
[2 8] (70398614, 70399506)
[2 9] (70398614, 70399506)
[ 2 10] (70398614, 70399506)
[ 2 12] (70398614, 70399597)
[ 2 15] (70398614, 70402838)
[ 2 16] (70398614, 70402838)
[ 2 17] (70398614, 70402844)
[ 2 18] (70398614, 70403210)
[ 2 19] (70398614, 70405264)
[ 2 20] (70398614, 70405264)
[ 2 21] (70398614, 70405264)
[ 2 22] (70398614, 70405265)
[ 2 23] (70398614, 704

In [111]:
# TEST
# ME (EDGE) 
#[170 203] (70435040, 70436454)
#[170 204] (70435040, 70436454)

In [34]:
for i, j in zip(seg_graph_index, segms_gene):
    print(i, j)

[ 0 21] (70398064, 70405264)
[4 8] (70398614, 70399506)
[ 4 11] (70398614, 70399597)
[ 4 17] (70398614, 70402838)
[ 4 18] (70398614, 70402844)
[ 4 20] (70398614, 70403210)
[ 4 21] (70398614, 70405264)
[ 4 22] (70398614, 70405265)
[ 4 23] (70398614, 70405321)
[ 4 30] (70398614, 70407297)
[ 4 49] (70398614, 70409830)
[ 4 56] (70398614, 70414541)
[ 4 57] (70398614, 70414545)
[ 4 84] (70398614, 70419745)
[ 7 21] (70399506, 70405264)
[ 8 21] (70399578, 70405264)
[11 21] (70399681, 70405264)
[14 17] (70399781, 70402838)
[14 21] (70399781, 70405264)
[15 17] (70399976, 70402838)
[15 21] (70399976, 70405264)
[18 21] (70403012, 70405264)
[23 30] (70405359, 70407297)
[24 26] (70405361, 70405981)
[24 28] (70405361, 70406586)
[24 29] (70405361, 70407030)
[24 30] (70405361, 70407297)
[24 34] (70405361, 70407517)
[25 30] (70405406, 70407297)
[28 30] (70406694, 70407297)
[31 34] (70407381, 70407517)
[32 34] (70407384, 70407517)
[32 35] (70407384, 70407518)
[33 34] (70407387, 70407517)
[35 49] (7040758

In [83]:
# TEST
print([ i for i, foo in enumerate(zip(np.where(gene.segmentgraph.seg_edges)[0],
                                np.where(gene.segmentgraph.seg_edges)[1])) 
 if (foo[0] == 169) & (foo[1] == 186)]) # THIS IS THE OFFSET

[175]


In [112]:
# TEST
# ANDRE 169 (array([70435025, 70435040])) and 186 (array([70436454, 70436562])
# OBSERVED ME [169 186] (70435040, 70436454) .. THIS IS THE RIGHT "SEGMENT GRAPH JUNCTION"

In [36]:
df_junction_gene = pd.DataFrame(junction_gene, columns = ['junction_start', 'junction_end'])

# Count File

In [54]:
# Count file 
h5f = h5py.File(count_path, 'r')
print(h5f.keys())
for k in h5f.keys():
    print(k)
    print(h5f[k].shape)
    sample_id = [i for i, j in enumerate(h5f['strains']) if j.decode() == sample_name][0]
    edge_expr_sample = h5f['edges'][:, sample_id]


<KeysViewHDF5 ['edge_idx', 'edges', 'gene_ids_edges', 'gene_ids_segs', 'gene_names', 'seg_len', 'seg_pos', 'segments', 'strains']>
edge_idx
(1235358,)
edges
(1235358, 1102)
gene_ids_edges
(1235358, 1)
gene_ids_segs
(1438125, 1)
gene_names
(60609, 1)
seg_len
(1438125, 1)
seg_pos
(1438125, 1102)
segments
(1438125, 1102)
strains
(1102,)


In [68]:
# TEST
test_multi_index_segm = 32296

In [55]:
gene_hd5_id = [i for i, j in enumerate(h5f['gene_names'][:, 0]) if j.decode() == gene.name][0]
print('gene id is:', gene_hd5_id)

gene id is: 9803


In [56]:
edges_index_gene = [i for i, j in enumerate(h5f['gene_ids_edges'][:, 0]) if j == gene_hd5_id]
print('found', len(edges_index_gene), 'edges corresponding to the gene')
print(edges_index_gene)


found 187 edges corresponding to the gene
[220830, 220831, 220832, 220833, 220834, 220835, 220836, 220837, 220838, 220839, 220840, 220841, 220842, 220843, 220844, 220845, 220846, 220847, 220848, 220849, 220850, 220851, 220852, 220853, 220854, 220855, 220856, 220857, 220858, 220859, 220860, 220861, 220862, 220863, 220864, 220865, 220866, 220867, 220868, 220869, 220870, 220871, 220872, 220873, 220874, 220875, 220876, 220877, 220878, 220879, 220880, 220881, 220882, 220883, 220884, 220885, 220886, 220887, 220888, 220889, 220890, 220891, 220892, 220893, 220894, 220895, 220896, 220897, 220898, 220899, 220900, 220901, 220902, 220903, 220904, 220905, 220906, 220907, 220908, 220909, 220910, 220911, 220912, 220913, 220914, 220915, 220916, 220917, 220918, 220919, 220920, 220921, 220922, 220923, 220924, 220925, 220926, 220927, 220928, 220929, 220930, 220931, 220932, 220933, 220934, 220935, 220936, 220937, 220938, 220939, 220940, 220941, 220942, 220943, 220944, 220945, 220946, 220947, 220948, 22094

In [57]:
segm_index_gene = [i for i, j in enumerate(h5f['gene_ids_segs'][:, 0]) if j == gene_hd5_id]
print('found', len(edges_index_gene), 'segments corresponding to the gene')
print(segm_index_gene)


found 187 segments corresponding to the gene
[249675, 249676, 249677, 249678, 249679, 249680, 249681, 249682, 249683, 249684, 249685, 249686, 249687, 249688, 249689, 249690, 249691, 249692, 249693, 249694, 249695, 249696, 249697, 249698, 249699, 249700, 249701, 249702, 249703, 249704, 249705, 249706, 249707, 249708, 249709, 249710, 249711, 249712, 249713, 249714, 249715, 249716, 249717, 249718, 249719, 249720, 249721, 249722, 249723, 249724, 249725, 249726, 249727, 249728, 249729, 249730, 249731, 249732, 249733, 249734, 249735, 249736, 249737, 249738, 249739, 249740, 249741, 249742, 249743, 249744, 249745, 249746, 249747, 249748, 249749, 249750, 249751, 249752, 249753, 249754, 249755, 249756, 249757, 249758, 249759, 249760, 249761, 249762, 249763, 249764, 249765, 249766, 249767, 249768, 249769, 249770, 249771, 249772, 249773, 249774, 249775, 249776, 249777, 249778, 249779, 249780, 249781, 249782, 249783, 249784, 249785, 249786, 249787, 249788, 249789, 249790, 249791, 249792, 249793, 24

In [88]:
# THIS CONTAINS THE MULTI-INDEX OF THE SEGM_EDGES which are not NULL IN THE SEGMENT GRAPH
edges_index_multi = h5f['edge_idx'][edges_index_gene] 
#print('found the following edge idx...', edges_index_gene_struct)
edges_index_multi = [int(i) for i in edges_index_multi]
print('found the following edge idx...', edges_index_multi)

found the following edge idx... [21, 768, 771, 777, 778, 780, 781, 782, 783, 790, 809, 816, 817, 844, 1351, 1541, 2111, 2677, 2681, 2867, 2871, 3441, 4400, 4586, 4588, 4589, 4590, 4594, 4780, 5350, 5924, 6114, 6115, 6304, 6699, 6889, 7070, 7071, 7074, 7079, 7163, 7269, 8789, 8790, 8792, 8796, 8797, 8979, 8980, 8982, 8986, 8987, 9360, 9362, 9363, 9366, 9367, 9375, 9377, 10126, 10127, 10506, 11104, 11272, 11461, 11462, 11463, 11465, 11467, 11469, 11522, 11552, 12035, 12037, 12038, 12039, 12054, 12068, 13194, 13384, 13565, 13566, 13570, 13573, 13574, 13588, 13608, 13756, 13764, 13798, 13954, 14144, 15284, 16046, 16628, 16629, 16631, 16632, 16648, 16649, 16652, 16663, 16818, 16821, 16838, 17388, 17408, 17768, 18911, 18912, 18924, 18925, 18928, 18929, 18932, 19484, 19688, 19878, 20068, 20258, 20259, 20260, 20638, 20642, 20828, 20832, 21018, 21968, 21969, 22158, 22159, 22921, 22922, 22923, 23503, 23685, 23687, 23689, 23692, 23693, 24263, 25213, 25605, 25796, 25984, 26553, 26745, 26937, 28078

In [113]:
# TEST 
absolute_pos = [myid for myid, edge_idx in enumerate(h5f['edge_idx']) if edge_idx == test_multi_index_segm][0]
print(absolute_pos)


221005


In [89]:
# TEST 
offset = [myid for myid, edge_idx in enumerate(h5f['edge_idx'][edges_index_gene]) if edge_idx == test_multi_index_segm][0]
print(offset)
print(h5f['edge_idx'][edges_index_gene][offset])

175
32296.0


In [ ]:
# TEST
# ME: THE MULTI INDEX 32296 CORRESPONDING TO THE SEGMENT NUMBER 169-186 IS FOUND in edge_idx and its edge IDX is 175

In [96]:
edges_index_gene_struct = None

In [98]:
print(len(set(segs_graph_index_flattened)))
print(len(set(edges_index_multi)))
print(len(set(segs_graph_index_flattened).intersection(set(edges_index_multi))))
print(len(set(segs_graph_index_flattened).difference(set(edges_index_multi))))
print(len(set(edges_index_multi).difference(set(segs_graph_index_flattened))))

187
187
187
0
0


In [100]:
print(edges_index_multi[0:10]) # From the count file
print(edges_graph_index_flattened[0:10])
print(segs_graph_index_flattened[0:10]) # From the graph


[21, 768, 771, 777, 778, 780, 781, 782, 783, 790]
[ 20 213 214 215 217 220 221 222 223 224]
[ 21 768 771 777 778 780 781 782 783 790]


In [129]:
# TEST absolute_pos
print(edge_expr_sample[absolute_pos])
print ('amoung which non-null', sum( h5f['edges'][absolute_pos, :]  > 0))
print ('which non-null', np.where( h5f['edges'][absolute_pos, :]  > 0))

0.0
amoung which non-null 1
which non-null (array([855]),)


In [158]:
print(len(edges_index_gene))

edge_expr_sample_gene = edge_expr_sample[edges_index_gene]
print('found following edge expression with first set of IDS \n ', edge_expr_sample_gene)
print ('amoung which non-null', sum(edge_expr_sample_gene > 0))




187
found following edge expression with first set of IDS 
  [  0.   0.   0.   0.   0.   0. 191.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   4.   0.   0.   6.   0.   0.   0.   0. 266.   0.
   0.   0.   0. 377.   0.   0.   0.   0.   0.   0.   0. 436.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   2.   3.   0. 424.
  11.   0.   0.   0.   6.   0.   0.   0.   0. 513.   0.   0.   0.   0.
   0.   0.   0. 493.   0.   0.   4.   0.   0.   0.   0.   0.   0.   0.
 600.   0.   0.   0.  16.   0.   0.   0.   0.   0. 644.   0.   0.   0.
   5.   0.   0.   0.   0.   0.   0.   0.   0.   0. 415.   0.   0.   0.
 162.   0.   0.   0.   0.   0.   0. 565.   0.   0.   0.   0.   0.   0.
   0.  11.   0.  15.   0.   0. 557.   0.   0.   0.   0.   0.   0. 815.
   0.   0.   0.   0.   0.   0.   0.   0. 451.   0.   0.   0. 365.   0.
   0.   0.   0.   0.   0.   0.   0.   0. 525.   0.   0.   0.   2.   0.
   0.   0.   0.   0. 445.   0.   0.   0.  10.   0.   0.   0.   0.   0.
   2.   0.   0. 

In [221]:
expressed_junc_indexes = np.where(edge_expr_sample[edges_index_gene])[0] 
print(expressed_junc_indexes)
#This should also be the offsets of interest

[  6  18  21  26  31  39  52  53  55  56  60  65  73  76  84  88  94  98
 108 112 119 127 129 132 139 148 152 162 166 172 176 182 186]


In [222]:
junctions_expressions = edge_expr_sample_gene[expressed_junc_indexes]

In [223]:
expressed_edges_index_multi = np.array(edges_index_multi)[expressed_junc_indexes] 
print(expressed_edges_index_multi)


[  781  2681  3441  4590  6114  7079  9360  9362  9366  9367 10127 11462
 12037 12054 13574 13764 16628 16648 18911 18928 20258 21968 22158 22922
 23693 28078 28652 29799 29809 30949 32479 34195 34575]


In [224]:
expressed_segm_ids = []
counter = 0
for index, multi_index in zip(seg_graph_index, segs_graph_index_flattened):
    if counter in expressed_junc_indexes:
        print(index, multi_index)
        expressed_segm_ids.append(index)
    counter+=1


[ 4 21] 781
[14 21] 2681
[18 21] 3441
[24 30] 4590
[32 34] 6114
[37 49] 7079
[49 50] 9360
[49 52] 9362
[49 56] 9366
[49 57] 9367
[53 57] 10127
[60 62] 11462
[63 67] 12037
[63 84] 12054
[71 84] 13574
[72 84] 13764
[87 98] 16628
[ 87 118] 16648
[ 99 101] 18911
[ 99 118] 18928
[106 118] 20258
[115 118] 21968
[116 118] 22158
[120 122] 22922
[124 133] 23693
[147 148] 28078
[150 152] 28652
[156 159] 29799
[156 169] 29809
[162 169] 30949
[170 179] 32479
[179 185] 34195
[181 185] 34575


In [225]:
expressed_segm_ids = []
counter = 0
for index, multi_index in zip(seg_graph_index, segs_graph_index_flattened):
    if multi_index in expressed_edges_index_multi:
        print(index, multi_index)
        expressed_segm_ids.append(index)

[ 4 21] 781
[14 21] 2681
[18 21] 3441
[24 30] 4590
[32 34] 6114
[37 49] 7079
[49 50] 9360
[49 52] 9362
[49 56] 9366
[49 57] 9367
[53 57] 10127
[60 62] 11462
[63 67] 12037
[63 84] 12054
[71 84] 13574
[72 84] 13764
[87 98] 16628
[ 87 118] 16648
[ 99 101] 18911
[ 99 118] 18928
[106 118] 20258
[115 118] 21968
[116 118] 22158
[120 122] 22922
[124 133] 23693
[147 148] 28078
[150 152] 28652
[156 159] 29799
[156 169] 29809
[162 169] 30949
[170 179] 32479
[179 185] 34195
[181 185] 34575


In [226]:
expressed_junctions_coord = [(gene.segmentgraph.segments[1][i], gene.segmentgraph.segments[0][j]) 
              for i, j in expressed_segm_ids]

In [227]:
for jx, expr in zip(expressed_junctions_coord, junctions_expressions):
    print(jx[0], jx[1], expr)

70398614 70405264 191.0
70399781 70405264 4.0
70403012 70405264 6.0
70405361 70407297 266.0
70407384 70407517 377.0
70407591 70409830 436.0
70409960 70410299 2.0
70409960 70413864 3.0
70409960 70414541 424.0
70409960 70414545 11.0
70414003 70414545 6.0
70414652 70415662 513.0
70415717 70417012 493.0
70415717 70419745 4.0
70417123 70419745 600.0
70417140 70419745 16.0
70419856 70420399 644.0
70419856 70422939 5.0
70420510 70421469 415.0
70420510 70422939 162.0
70421580 70422939 565.0
70422592 70422939 11.0
70422631 70422939 15.0
70422995 70425331 557.0
70425401 70429050 815.0
70429199 70431190 451.0
70431280 70433100 365.0
70433278 70433646 525.0
70433278 70435025 2.0
70433718 70435025 445.0
70435119 70435465 10.0
70435555 70436281 2.0
70435744 70436281 53.0


In [228]:
expressed_df_graph = [[jx[0], jx[1], expr] 
                      for jx, expr in zip(expressed_junctions_coord, junctions_expressions)]


In [229]:
expressed_df_graph = pd.DataFrame(expressed_df_graph, columns = ['junction_start', 'junction_end', 'count_graph'])
expressed_df_graph = expressed_df_graph.drop_duplicates()

In [230]:
expressed_df_graph.head()

,junction_start,junction_end,count_graph
0,70398614,70405264,191.0
1,70399781,70405264,4.0
2,70403012,70405264,6.0
3,70405361,70407297,266.0
4,70407384,70407517,377.0


# EXPRESSION OF ALL JUNCTIONS IN THE GENE

In [231]:
# ALL 
segm_ids_with_expression = []
for index, multi_index in zip(seg_graph_index, segs_graph_index_flattened):
    if multi_index in edges_index_multi:
        print(index, multi_index)
        segm_ids_with_expression.append(index)

[ 0 21] 21
[4 8] 768
[ 4 11] 771
[ 4 17] 777
[ 4 18] 778
[ 4 20] 780
[ 4 21] 781
[ 4 22] 782
[ 4 23] 783
[ 4 30] 790
[ 4 49] 809
[ 4 56] 816
[ 4 57] 817
[ 4 84] 844
[ 7 21] 1351
[ 8 21] 1541
[11 21] 2111
[14 17] 2677
[14 21] 2681
[15 17] 2867
[15 21] 2871
[18 21] 3441
[23 30] 4400
[24 26] 4586
[24 28] 4588
[24 29] 4589
[24 30] 4590
[24 34] 4594
[25 30] 4780
[28 30] 5350
[31 34] 5924
[32 34] 6114
[32 35] 6115
[33 34] 6304
[35 49] 6699
[36 49] 6889
[37 40] 7070
[37 41] 7071
[37 44] 7074
[37 49] 7079
[ 37 133] 7163
[38 49] 7269
[46 49] 8789
[46 50] 8790
[46 52] 8792
[46 56] 8796
[46 57] 8797
[47 49] 8979
[47 50] 8980
[47 52] 8982
[47 56] 8986
[47 57] 8987
[49 50] 9360
[49 52] 9362
[49 53] 9363
[49 56] 9366
[49 57] 9367
[49 65] 9375
[49 67] 9377
[53 56] 10126
[53 57] 10127
[55 56] 10506
[58 84] 11104
[59 62] 11272
[60 61] 11461
[60 62] 11462
[60 63] 11463
[60 65] 11465
[60 67] 11467
[60 69] 11469
[ 60 122] 11522
[ 60 152] 11552
[63 65] 12035
[63 67] 12037
[63 68] 12038
[63 69] 12039
[63 84

In [232]:
for jx, expr in zip(segm_ids_with_expression, edge_expr_sample_gene):
    print(jx[0], jx[1], expr)

0 21 0.0
4 8 0.0
4 11 0.0
4 17 0.0
4 18 0.0
4 20 0.0
4 21 191.0
4 22 0.0
4 23 0.0
4 30 0.0
4 49 0.0
4 56 0.0
4 57 0.0
4 84 0.0
7 21 0.0
8 21 0.0
11 21 0.0
14 17 0.0
14 21 4.0
15 17 0.0
15 21 0.0
18 21 6.0
23 30 0.0
24 26 0.0
24 28 0.0
24 29 0.0
24 30 266.0
24 34 0.0
25 30 0.0
28 30 0.0
31 34 0.0
32 34 377.0
32 35 0.0
33 34 0.0
35 49 0.0
36 49 0.0
37 40 0.0
37 41 0.0
37 44 0.0
37 49 436.0
37 133 0.0
38 49 0.0
46 49 0.0
46 50 0.0
46 52 0.0
46 56 0.0
46 57 0.0
47 49 0.0
47 50 0.0
47 52 0.0
47 56 0.0
47 57 0.0
49 50 2.0
49 52 3.0
49 53 0.0
49 56 424.0
49 57 11.0
49 65 0.0
49 67 0.0
53 56 0.0
53 57 6.0
55 56 0.0
58 84 0.0
59 62 0.0
60 61 0.0
60 62 513.0
60 63 0.0
60 65 0.0
60 67 0.0
60 69 0.0
60 122 0.0
60 152 0.0
63 65 0.0
63 67 493.0
63 68 0.0
63 69 0.0
63 84 4.0
63 98 0.0
69 84 0.0
70 84 0.0
71 75 0.0
71 76 0.0
71 80 0.0
71 83 0.0
71 84 600.0
71 98 0.0
71 118 0.0
72 76 0.0
72 84 16.0
72 118 0.0
73 84 0.0
74 84 0.0
80 84 0.0
84 86 0.0
87 98 644.0
87 99 0.0
87 101 0.0
87 102 0.0
87 118 5.0

In [235]:
all_junctions_coord_with_expr = [(gene.segmentgraph.segments[1][i], gene.segmentgraph.segments[0][j]) 
              for i, j in segm_ids_with_expression]

In [236]:
all_junctions_coord_with_expr = [[jx[0], jx[1], expr] 
                      for jx, expr in zip(all_junctions_coord_with_expr, edge_expr_sample_gene)]


In [237]:
all_expr_df_graph = pd.DataFrame(all_junctions_coord_with_expr, columns = ['junction_start', 'junction_end', 'count_graph'])
all_expr_df_graph = all_expr_df_graph.drop_duplicates()

In [242]:
all_expr_df_graph.loc[all_expr_df_graph['count_graph'] != 0 ]

,junction_start,junction_end,count_graph
6,70398614,70405264,191.0
18,70399781,70405264,4.0
21,70403012,70405264,6.0
26,70405361,70407297,266.0
31,70407384,70407517,377.0
39,70407591,70409830,436.0
52,70409960,70410299,2.0
53,70409960,70413864,3.0
55,70409960,70414541,424.0
56,70409960,70414545,11.0


In [243]:
all_junctions_coord_with_expr_quick = [[jx[0], jx[1], expr] 
                      for jx, expr in zip(seg_graph_index, edge_expr_sample_gene)]


In [244]:
all_expr_df_graph_quick = pd.DataFrame(all_junctions_coord_with_expr_quick, columns = ['junction_start', 'junction_end', 'count_graph'])
all_expr_df_graph_quick = all_expr_df_graph.drop_duplicates()

In [245]:
all_expr_df_graph_quick.equals(all_expr_df_graph)

True

In [246]:
# CLOSE
h5f.close()

# MORE TEST SINGLE GENE MERGE

In [213]:
junction_graph_unfilt = expressed_df_graph.merge(df_jx_unfiltered, on = ['junction_start', 'junction_end'], how = 'left')

In [214]:
display(junction_graph_unfilt.head())
print(junction_graph_unfilt.shape)
print(junction_graph_unfilt.drop_duplicates().shape)
#junction_graph_unfilt = junction_graph_unfilt.merge(df_jx_unfiltered, on = ['junction_start', 'junction_end'], how = 'left')

,junction_start,junction_end,count_graph,chr,strand,count
0,70398614,70405264,191.0,chr11,+,191
1,70399781,70405264,4.0,chr11,+,4
2,70403012,70405264,6.0,chr11,+,6
3,70405361,70407297,266.0,chr11,+,266
4,70407384,70407517,377.0,chr11,+,377


(33, 6)
(33, 6)


In [216]:
junction_graph_unfilt

,junction_start,junction_end,count_graph,chr,strand,count
0,70398614,70405264,191.0,chr11,+,191
1,70399781,70405264,4.0,chr11,+,4
2,70403012,70405264,6.0,chr11,+,6
3,70405361,70407297,266.0,chr11,+,266
4,70407384,70407517,377.0,chr11,+,377
5,70407591,70409830,436.0,chr11,+,436
6,70409960,70410299,2.0,chr11,+,2
7,70409960,70413864,3.0,chr11,+,3
8,70409960,70414541,424.0,chr11,+,424
9,70409960,70414545,11.0,chr11,+,11


# Graph all genes

In [ ]:
graph, meta = pickle.load(open(splice_path, 'rb'))

In [ ]:
junction_all_genes = []
for idx, gene in enumerate(graph):
    if idx % 1000 == 0:
        print(idx)
    gene.from_sparse()
    edges_coords = np.array([(a,b) for a, b in zip(np.where(gene.splicegraph.edges)[0],np.where(gene.splicegraph.edges)[1])])
    junction_gene = [(gene.splicegraph.vertices[1][i], gene.splicegraph.vertices[0][j]) for i, j in edges_coords]
    junction_all_genes.extend(junction_gene)

In [ ]:
df_junction_gene = pd.DataFrame(junction_all_genes, columns = ['junction_start', 'junction_end'])


In [ ]:
df_junction_gene.head()

In [ ]:
df_junction_gene.shape

In [ ]:
df_junction_gene.drop_duplicates().shape

In [ ]:
jxS_graph_unfilt = df_junction_gene.drop_duplicates().merge(\
    df_jx_unfiltered.loc[:, ['junction_start', 'junction_end']].drop_duplicates(), 
                                          on = ['junction_start', 'junction_end'], how = 'inner')

In [ ]:
jxS_graph_unfilt.drop_duplicates().shape